In [ ]:
# Setting run equal to false.  If set to true, code will pull all matches for
# user in the gamertag variable
run = False


if run == True:
    # Setting gamertag, API key, 'start' (0 means most recent match played)
    gamertag = 'Drymander'
    headers = {
        # Request headers
        'Ocp-Apim-Subscription-Key': api_key,
    }
    start = 0
    matches = []
    
    # Condition to determine whether or not API call has exhausted full match history
    maxed_out = False
    
    # While loop that wil continuously pull from API and terminate once complete
    # When 'count' returns as 25, that means there are more matches to be pulled
    # When 'count' is less than 25, that indicates it is at the end of the player match history
    while maxed_out == False:
        try:
            params = urllib.parse.urlencode({
                # Request parameters
                'modes': 'arena',
                'start': start,
                'count': 25,
                'include-times': True,
            })
            # Code to process the API byte string into a JSON 
            # Will append each JSON to a list
            conn = http.client.HTTPSConnection('www.haloapi.com')
            conn.request("GET", f"/stats/h5/players/{gamertag}/matches?%s" % params, "{body}", headers)
            response = conn.getresponse()
            data = response.read()
            data = json.loads(data)
            matches.extend(data['Results'])
            conn.close()
            
            # If statement to detect final API call
            if data['ResultCount'] == 25:
                maxed_out = False
                time.sleep(1.1)
                start += 25
            else:
                maxed_out = True
        
        # Error handling
        except Exception as e:
            print("error")
    
    # Save to dataframe and CSV
    df = pd.DataFrame(matches)
    df.to_csv('DrymanderHistory.csv')



In [ ]:
# Loading from copy
df = df_predate.copy()

# Setting seaborn styles
sns.set(rc={'axes.facecolor':'white'})
sns.set_style("darkgrid")
sns.set_context('talk')

# Creating 'Month' column
df['Month'] = pd.to_datetime(df['Date']).dt.to_period('M')
df['Month'] = df['Month'].dt.strftime('%B, %Y')

# Setting time frame of data to visualize
df = df[(df['Date'] > '2020-05-01')]

# Instantiating graph and countplot
f, ax = plt.subplots(figsize=(16, 9))
ax = sns.countplot(x="Month", 
                   data=df.sort_values(by=['Date'], ascending=True), 
                   hue='WinLoseTie',
                   hue_order=['Victory','Defeat','Tie'])

# Setting xtick labels, axis titles, plot titles, and legend order
ax.set_xticklabels(ax.get_xticklabels(),rotation=45, horizontalalignment='right');
ax.set_title('Wins, Losses, and Ties by Month', fontsize=30)
ax.set(ylabel='Count')
plt.legend(labels=['Victory', 'Defeat', 'Tie'])

# Reverting back to original df for convenience with the next visualization
df = df_predate.copy()

In [ ]:
# Function to pull most recent match stats into JSON format
# Uses two separate API calls, one from player history and another from match details
def pull_recent_match(how_recent, api_key=api_key, explore=False, gamertag='Drymander'):
    
    # Use gamertag_for_api function to remove any spaces
    gamertag = gamertag_for_api(gamertag)
    headers = {
        # Request headers
        'Ocp-Apim-Subscription-Key': api_key,
    }
    # Pulls from arena mode, how_recent is how far to go back in the match history
    # 'count' refers to the number of matches to pull
    params = urllib.parse.urlencode({
        # Request parameters
        'modes': 'arena',
        'start': how_recent,
        'count': 1,
        'include-times': True,
    })
    
    # Try this, otherwise return error message
    try:
        
        # Connect to API and pull most recent match for specified gamer
        conn = http.client.HTTPSConnection('www.haloapi.com')
        conn.request("GET", f"/stats/h5/players/{gamertag}/matches?%s" % params, "{body}", headers)
        response = conn.getresponse()
        latest_match = json.loads(response.read())
        
        # Identify match ID and match date
        match_id = latest_match['Results'][0]['Id']['MatchId']
        match_date = latest_match['Results'][0]['MatchCompletedDate']['ISO8601Date']
        
        # Rest for 1.01 seconds to not get blocked by API
        time.sleep(1.01)
        
        # Using match_id, pull details from match
        conn.request("GET", f"/stats/h5/arena/matches/{match_id}?%s" % params, "{body}", headers)
        response = conn.getresponse()
        data = response.read()
        
        # Option to return as byte string for alternative viewing
        if explore == True:
            print(data)
        else:
            # Append match ID and date from player history API call
            match_results = json.loads(data)
            match_results['MatchId'] = match_id
            match_results['Date'] = match_date
        conn.close()
    
    # Print error if issue with calling API
    except Exception as e:
        print(f"[Errno {0}] {1}".format(e.errno, e.strerror))
    
    # Return match results as JSON
    return match_results

# Show result
match_results = pull_recent_match(0, explore=False, gamertag='Drymander')
match_results

In [ ]:
# Function that combines all functions above to go through each step to
# Get the match dataframe
def recent_match_stats(gamertag, back_count=0):
    
    # Pull the match result as JSON from API
    match_results = pull_recent_match(back_count, explore=False, gamertag=gamertag)
    
    # Build the base dataframe
    base_df = build_base_dataframe(match_results, gamertag=gamertag)
    
    # Sleep for 1.01 seconds to avoid issues with API
    time.sleep(1.01)
    
    # Create playerlist for player history API call
    player_list = get_player_list(base_df)
    
    # Call API to get player history JSON
    player_history = get_player_history(base_df)
    
    # Build base player stats dataframe based on player history API call
    history_df = build_history_dataframe(player_history, match_results['GameBaseVariantId'])
    
    # Merge the base dataframe and stats dataframe
    full_stats_df = pd.merge(base_df, history_df, how='inner', on = 'Gamertag')
    
    return full_stats_df

# Show full dataframe for match
df = recent_match_stats('Drymander', back_count=0)
df